### **Karatsuba Formula in GF(2)**

Given $ A(x), B(x) $, split at $ m = \lceil \deg/2 \rceil $:


\begin{aligned}
A_0 &= A \bmod x^m, \quad &A_1 &= \lfloor A / x^m \rfloor \\
B_0 &= B \bmod x^m, \quad &B_1 &= \lfloor B / x^m \rfloor \\
z_0 &= A_0 \cdot B_0 \\
z_2 &= A_1 \cdot B_1 \\
z_1 &= (A_0 + A_1) \cdot (B_0 + B_1) \\
C(x) &= z_2 \cdot x^{2m} \oplus (z_1 \oplus z_0 \oplus z_2) \cdot x^m \oplus z_0
\end{aligned}


where $ \oplus $ denotes coefficient-wise XOR (i.e., addition in GF(2)).

---

### **Example**

Let $ A(x) = x^3 + x + 1  \to$ binary `1011` → integer `11`  
Let $ B(x) = x^2 + 1 \to$ binary `0101` → integer `5`

Split at $ m = 2 $:
- $ A_0 = x + 1 $ (`11` = 3), $ A_1 = x $ (`10` = $2$)
- $ B_0 = 1 $ (`01` = 1), $ B_1 = 1 $ (`01` = $1$)

Compute:
- $ z_0 = A_0 \cdot B_0 = 3 \cdot 1 = x + 1 $
- $ z_2 = A_1 \cdot B_1 = 2 \cdot 1 = x $
- $ A_0 + A_1 = (x+1) + x = 1 $
- $ B_0 + B_1 = 1 + 1 = 0 $
- $ z_1 = (A_0 + A_1) \cdot (B_0 + B_1) = 1 \cdot (0) = 0 $
- $ z_1 \oplus z_0 \oplus z_2 = 0 \oplus (x+1) \oplus x = 1 $

Then:
\begin{aligned}
C(x) &= z_2 \cdot x^{2m} \oplus (z_1 \oplus z_0 \oplus z_2) \cdot x^m \oplus z_0 \\
     &= x \cdot x^4 \oplus 1 \cdot x^2 \oplus (x+1) = x^5 \oplus x^2 \oplus x \oplus 1
\end{aligned}

Which matches $ (x^3+x+1)(x^2+1) = x^5 + x^2 + x + 1 $.

(For verification: use your `schoolbook_mul64` — it will match.)

---

This is the **exact mathematical foundation** for recursive or iterative Karatsuba GF(2) multiplier.

In [2]:
def schoolbook_mul64(a, b):
    """64x64 --> 128-bit GF(2) multiplication. Returns integer (not limbs)."""
    a &= (1 << 64) - 1
    b &= (1 << 64) - 1
    low = 0
    high = 0
    for i in range(64):
        if (a >> i) & 1:
            if i == 0:
                low ^= b
            else:
                low ^= b << i
                high ^= b >> (64 - i)
    return low | (high << 64)

In [6]:
def karatsuba_gf2_recursive(a, b):
    """
    Recursively multiply two GF(2) polynomials (represented as Python ints).
    Uses schoolbook_mul64 when both a and b fit in 64 bits.
    """
    # Base case: both operands fit in 64 bits
    if a < (1 << 64) and b < (1 << 64):
        print(a)
        return schoolbook_mul64(a, b)

    # Ensure a is the longer operand (optional optimization)
    if a.bit_length() < b.bit_length():
        a, b = b, a

    # If b is zero, return zero
    if b == 0:
        return 0

    # Split a around its midpoint
    n = a.bit_length()
    m = (n + 1) // 2  # ceiling(n/2)

    # Split: a = a1 * x^m + a0
    a0 = a & ((1 << m) - 1)
    a1 = a >> m

    # Split b similarly (even if b is shorter)
    b0 = b & ((1 << m) - 1)
    b1 = b >> m

    # Three recursive products
    z0 = karatsuba_gf2_recursive(a0, b0)          # a0 * b0
    z2 = karatsuba_gf2_recursive(a1, b1)          # a1 * b1

    # (a0 + a1) and (b0 + b1) → XOR in GF(2)
    a0_a1 = a0 ^ a1
    b0_b1 = b0 ^ b1

    z1 = karatsuba_gf2_recursive(a0_a1, b0_b1)    # (a0+a1)*(b0+b1)
    z1 ^= z0 ^ z2                                 # z1 = z1 - z0 - z2 → XOR in GF(2)

    # Combine: z2 * x^(2m) + z1 * x^m + z0
    return (z2 << (2 * m)) ^ (z1 << m) ^ z0

In [11]:
# Example: two 17669-bit polynomials
BIT_WIDTH = 17669

a = 1235453908304758023475342453      # x^17668 + 12345
b = 1254043975983457034753532453  # sparse polynomial

# Multiply using recursive Karatsuba
product = karatsuba_gf2_recursive(a, b)

print(product)

12575548105253
17821036749175
30076339176274
1522353775734672659161729858761382190419002801376139521


In [12]:
len(bin(a)[2:])

90

verification

In [13]:
import galois

GF2 = galois.GF(2)

def gf2_mul_to_int(a: int, b: int) -> int:
    pa = galois.Poly.Int(a, field=GF2)  # Correctly interprets bits as poly coeffs
    pb = galois.Poly.Int(b, field=GF2)
    pc = pa * pb
    return int(pc)

# Test
result = gf2_mul_to_int(a, b)  # 8 = x^3, so x^3 * x^3 = x^6 = 64
print(f"{a} * {b} in GF(2) = {result}")  # Output: 17

1235453908304758023475342453 * 1254043975983457034753532453 in GF(2) = 1522353775734672659161729858761382190419002801376139521
